In [1]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np
from tpot import TPOTClassifier

# Read in data and display first 5 rows
z_data = pd.read_csv('data_task_groups_roi_all.csv',sep=';',decimal='.')


   
# recode labels to numeric values so t_pot can handle them
# alphabetically: 0: C, 1: R, 2, S
z_data['task'] = z_data['task'].replace({'C': 0, 'R': 1, 'S': 2})

numLabels = 3 # num of different labels (comprehension and rest)
sizeTrainSet = 12 # num of participants used for training
sizeTestSet = 4 # num participants used for testing
numParticipants = sizeTrainSet + sizeTestSet

In [2]:
#first, drop everything non-numeric ===========================================================
z_data = z_data.drop('scan', axis = 1)
z_data = z_data.drop('trial', axis = 1)
z_data = z_data.drop('response', axis = 1)

In [4]:
#group the data and get the labels after the grouping
#return the grouped data and the labels
def prepare(data,groupingColumns):
    grouped = data.groupby(groupingColumns)
    groupedAgg = grouped.aggregate(np.mean)
    labels = groupedAgg.index.get_level_values(level='task')

    return groupedAgg,labels

In [5]:
#split according to participants (first 12 and last 4)
#return all four sets (much like train_test_split from sklearn)
#TODO: introduce variation (see file coarseAverageParticipantSplit)

def split(features, labels, low, high):
    #first, drop everything non-numeric ===========================================================
    
    #=== creating training and validation set ======================================================
    # Saving feature names for later use
    feature_list = list(features.columns)

    # Convert to numpy array
    features = np.array(features)
    labels = np.array(labels)

    #get the rows excluding the last indexed (i.e., row with the index 587 is the last to include)
    training_features = features[0:low]
    training_features = np.array(training_features)
    testing_features = features[low:high]
    testing_features = np.array(testing_features)

    training_target = labels[0:low]
    training_target = np.array(training_target)
    testing_target = labels[low:high]
    testing_target = np.array(testing_target)
    #=== end creating training and validation set ===================================================
    
    return training_features, testing_features, training_target, testing_target

In [6]:
def findIndices(name):
    numGroups = int(name.split('_')[1])
    low = sizeTrainSet*numGroups*numLabels
    high = numParticipants*numGroups*numLabels
    return low, high

In [7]:
#define the columns to group by
groupingColumn1 = 'proband'
groupingColumn2 = 'task'
groupingColumn3 = ''

#define how many different variants of aggregation and aggregation levels
numVariationsPerAggLevel = 10
numAggLevels = 40
numAggregationVariants = numVariationsPerAggLevel * numAggLevels

#get the index of the starting column that contains the labels of the groupings
offset = z_data.columns.get_loc('aggr_2_groups0')

collectAccuracies = []

#collect all instances where unmatched is not possible.
unmatched = []

#loop through the columns
#for each column, group the data set
# transform it to numpy.ndarray and split into training and validation set
# run t_pot
# and do the learning
while offset < len(z_data.columns):
    # group the data set
    #=============================================================================================================
    columnsForGrouping = [groupingColumn1,groupingColumn2,z_data.columns[offset]]
    features, labels = prepare(z_data,columnsForGrouping)
    #=============================================================================================================

    #=============================================================================================================
    # transform to numpy.ndarray and do split
    # find low and high
    low, high = findIndices(z_data.columns[offset])
    training_features, testing_features, training_target, testing_target = split(features, labels, low, high)
    #=============================================================================================================
    
    folder = 'allROIS/'
    fileName = 'tpot_mnist_pipeline_triangulateAggregationLevelParticipantSplit' + z_data.columns[offset] + '.py'

    #=============================================================================================================
    # run t_pot
    # TODO: extract method
    tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2,n_jobs=20)
    tpot.fit(training_features, training_target)
    print(tpot.score(testing_features, testing_target))
    tpot.export(folder + fileName)
    #=============================================================================================================    
        
    offset = offset + 1

Optimization Progress:  33%|███▎      | 40/120 [00:07<01:49,  1.37s/pipeline]

Generation 1 - Current best internal CV score: 0.9566666666666667


Optimization Progress:  50%|█████     | 60/120 [00:10<01:00,  1.01s/pipeline]

Generation 2 - Current best internal CV score: 0.9566666666666667


Optimization Progress:  68%|██████▊   | 81/120 [05:13<59:08, 90.99s/pipeline]  

...

Optimization Progress:  67%|██████▋   | 80/120 [02:06<11:16, 16.92s/pipeline]

Generation 3 - Current best internal CV score: 0.8384126984126985


Optimization Progress:  83%|████████▎ | 100/120 [02:26<04:39, 13.98s/pipeline]

Generation 4 - Current best internal CV score: 0.8384126984126985


Generation 5 - Current best internal CV score: 0.8428571428571429

Best pipeline: LogisticRegression(LogisticRegression(input_matrix, C=0.5, dual=True, penalty=l2), C=10.0, dual=True, penalty=l2)
0.8333333333333334


In [8]:
offset = z_data.columns.get_loc('aggr_2_groups0')

collectAccuracies = []

#collect all instances where unmatched is not possible.
unmatched = []

#loop through the columns
#for each column, group the data set
# transform it to numpy.ndarray and split into training and validation set
# run t_pot
# and do the learning
while offset < len(z_data.columns):
    # group the data set
    #=============================================================================================================
    columnsForGrouping = [groupingColumn1,groupingColumn2,z_data.columns[offset]]
    features, labels = prepare(z_data,columnsForGrouping)
    #=============================================================================================================

    #=============================================================================================================
    # transform to numpy.ndarray and do split
    # find low and high
    low, high = findIndices(z_data.columns[offset])
    training_features, testing_features, training_target, testing_target = split(features, labels, low, high)

    fileName = 'tpot_mnist_pipeline_triangulateAggregationLevelParticipantSplit' + z_data.columns[offset] + '.py'
    #=============================================================================================================
    #do the learning
    # TODO: extract method
    with open(folder + fileName) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    # content = [x.strip() for x in content] 

    #  or 'exported_pipeline = ' not in line
    cleanedContent = []
    for line in content:
        if 'tpot_data' not in line and 'training_target, testing_target' not in line:
            cleanedContent.append(line)
    #         content.remove(line)
    
    fileNameCleaned = folder + 'cleaned_' + fileName
    with open(fileNameCleaned, 'w') as filehandle:  
        for line in cleanedContent:
            filehandle.write('%s\n' % line)
            
    %run -i $fileNameCleaned
    
    # print the accuracy
    # TODO: extract method
    # TODO: print all accurracy values to one file
    num_matches = 0;
    for a, b in zip(testing_target, results):
        if a == b:
            num_matches = num_matches + 1
        else:
            unmatched.append((a,b))
    print('Number of matches:',num_matches,'(of',testing_target.size,')')

    accuracy = num_matches/testing_target.size*100
    print('Accuary: ',accuracy)
    #=============================================================================================================
    
    #=============================================================================================================
    #collect all accuracy values to plot them
    collectAccuracies.append(accuracy)

    offset = offset + 1

Number of matches: 22 (of 24 )
Accuary:  91.66666666666666
Number of matches: 24 (of 24 )
Accuary:  100.0
Number of matches: 23 (of 24 )
Accuary:  95.83333333333334
Number of matches: 21 (of 24 )
Accuary:  87.5
Number of matches: 23 (of 24 )
Accuary:  95.83333333333334
Number of matches: 24 (of 24 )
Accuary:  100.0
Number of matches: 23 (of 24 )
Accuary:  95.83333333333334
Number of matches: 23 (of 24 )
Accuary:  95.83333333333334
Number of matches: 18 (of 24 )
Accuary:  75.0
Number of matches: 22 (of 24 )
Accuary:  91.66666666666666
Number of matches: 36 (of 36 )
Accuary:  100.0
Number of matches: 32 (of 36 )
Accuary:  88.88888888888889
Number of matches: 34 (of 36 )
Accuary:  94.44444444444444
Number of matches: 34 (of 36 )
Accuary:  94.44444444444444
Number of matches: 33 (of 36 )
Accuary:  91.66666666666666
Number of matches: 33 (of 36 )
Accuary:  91.66666666666666
Number of matches: 32 (of 36 )
Accuary:  88.88888888888889


/users/author/ml_exec/venv1/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Number of matches: 34 (of 36 )
Accuary:  94.44444444444444
Number of matches: 34 (of 36 )
Accuary:  94.44444444444444
Number of matches: 30 (of 36 )
Accuary:  83.33333333333334


/users/author/ml_exec/venv1/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Number of matches: 41 (of 48 )
Accuary:  85.41666666666666
Number of matches: 44 (of 48 )
Accuary:  91.66666666666666
Number of matches: 42 (of 48 )
Accuary:  87.5
Number of matches: 40 (of 48 )
Accuary:  83.33333333333334
Number of matches: 42 (of 48 )
Accuary:  87.5
Number of matches: 42 (of 48 )
Accuary:  87.5
Number of matches: 45 (of 48 )
Accuary:  93.75
Number of matches: 42 (of 48 )
Accuary:  87.5
Number of matches: 39 (of 48 )
Accuary:  81.25
Number of matches: 43 (of 48 )
Accuary:  89.58333333333334
Number of matches: 58 (of 72 )
Accuary:  80.55555555555556
Number of matches: 59 (of 72 )
Accuary:  81.94444444444444
Number of matches: 55 (of 72 )
Accuary:  76.38888888888889
Number of matches: 57 (of 72 )
Accuary:  79.16666666666666
Number of matches: 59 (of 72 )
Accuary:  81.94444444444444
Number of matches: 58 (of 72 )
Accuary:  80.55555555555556


/users/author/ml_exec/venv1/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Number of matches: 58 (of 72 )
Accuary:  80.55555555555556
Number of matches: 63 (of 72 )
Accuary:  87.5
Number of matches: 62 (of 72 )
Accuary:  86.11111111111111
Number of matches: 60 (of 72 )
Accuary:  83.33333333333334


In [9]:
print(len(unmatched))
countComprehensionRest = 0
countComprehensionSyntax = 0
countRestSyntax = 0
for x in unmatched:
    if x == (0,1) or x == (1,0):
        countComprehensionRest = countComprehensionRest + 1
    if x == (0,2) or x == (2,0):
        countComprehensionSyntax = countComprehensionSyntax + 1
    if x == (1,2) or x == (2,1):
        countRestSyntax = countRestSyntax + 1
print('Comprehension/Rest',countComprehensionRest)
print('Comprehension/Syntax',countComprehensionSyntax)
print('Rest/Syntax',countRestSyntax)

236
Comprehension/Rest 25
Comprehension/Syntax 126
Rest/Syntax 85


In [10]:
#input: alist: the list to split, wanted_parts: into how many parts it should be split
#returns: all split lists
def split_list(alist, wanted_parts=1):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]


In [11]:
list1, list2, list3, list4 = split_list(collectAccuracies,4)
frames = [list1, list2, list3, list4]
collectedAccuracies = pd.DataFrame(frames).T

collectedAccuracies.columns = ['2_groups_allROIs','3_groups_allROIs','4_groups_allROIs','6_groups_allROIs']

collectedAccuracies.to_csv('triangulate_AllROIs.csv',sep=';',decimal='.', index=False)